In [117]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np
import gmaps.datasets

# Import API key
from config import g_key

# Configure gmaps
gmaps.configure(api_key=g_key)

In [118]:
# 1. Read the WeatherPy_vacation.csv into a DataFrame.
f = open(r"/Users/jacobnyamu/UCB Class Folder /World_Weather_Analysis/World_Weather_Analysis/Vacation Search/WeatherPy_Vacation.csv")

vacation_df = pd.read_csv(f)
vacation_df.head()

,City_ID,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,3,Ambilobe,MG,72.90,broken clouds,-13.2000,49.0500,Hôtel & Restaurant DIANA Ambilobe
1,4,Rikitea,PF,76.73,scattered clouds,-23.1203,-134.9692,People ThankYou
2,8,Banda Aceh,ID,78.08,overcast clouds,5.5577,95.3222,Sulthan Hotel International
3,13,Hithadhoo,MV,81.72,scattered clouds,-0.6000,73.0833,Scoop Guest House
4,15,Christchurch,NZ,82.60,few clouds,-43.5333,172.6333,The George


In [119]:
clean_vacation_df = vacation_df.dropna()
clean_vacation_df.isnull().sum()

City_ID                0
City                   0
Country                0
Max Temp               0
Current Description    0
Lat                    0
Lng                    0
Hotel Name             0
dtype: int64

In [120]:
# 2. Using the template add the city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 3a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_vacation_df.iterrows()]

# 3b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_vacation_df[["Lat", "Lng"]]

In [121]:
# 4a. Add a marker layer for each city to the map.
max_temp = clean_vacation_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# 4b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))

In [122]:
# From the map above pick 4 cities and create a vacation itinerary route to travel between the four cities. 
# 5. Create DataFrames for each city by filtering the 'vacation_df' using the loc method. 
# Hint: The starting and ending city should be the same city.

vacation_start = clean_vacation_df.loc[clean_vacation_df['City'] == 'Monrovia']
vacation_end = clean_vacation_df.loc[clean_vacation_df['City'] == 'Monrovia']
vacation_stop1 = clean_vacation_df.loc[clean_vacation_df['City'] == 'Tabou']
vacation_stop2 = clean_vacation_df.loc[clean_vacation_df['City'] == 'Axim']
vacation_stop3 = clean_vacation_df.loc[clean_vacation_df['City'] == 'Porto Novo']
vacation_start

,City_ID,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
98,268,Monrovia,LR,78.75,light rain,6.3005,-10.7969,Sunset Inn


In [123]:
vacation_stop1

,City_ID,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
119,324,Tabou,CI,78.35,light rain,4.423,-7.3528,hotêl doufoulougou


In [124]:
vacation_stop2

,City_ID,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
165,420,Axim,GH,78.69,overcast clouds,4.8699,-2.2405,Axim Beach Resort And Spa


In [125]:
vacation_stop3

,City_ID,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
159,406,Porto Novo,BJ,82.22,overcast clouds,6.4965,2.6036,Tour Eiffel Hotel Benin


In [126]:
# 6. Get the latitude-longitude pairs as tuples from each city DataFrame using the to_numpy function and list indexing.
start = vacation_start[['Lat','Lng']].to_numpy()[0]
end = vacation_end[['Lat','Lng']].to_numpy()[0]
stop1 = vacation_stop1[['Lat','Lng']].to_numpy()[0]
stop2 = vacation_stop2[['Lat','Lng']].to_numpy()[0]
stop3 = vacation_stop3[['Lat','Lng']].to_numpy()[0]
start, end, stop1, stop2, stop3

(array([  6.3005, -10.7969]),
 array([  6.3005, -10.7969]),
 array([ 4.423 , -7.3528]),
 array([ 4.8699, -2.2405]),
 array([6.4965, 2.6036]))

In [127]:
# 7. Create a direction layer map using the start and end latitude-longitude pairs,
# and stop1, stop2, and stop3 as the waypoints. The travel_mode should be "DRIVING", "BICYCLING", or "WALKING".
fig = gmaps.figure()
vacay_trip = gmaps.directions_layer(
        start, end, waypoints=[stop1,stop2,stop3],
        travel_mode='DRIVING')
fig.add_layer(vacay_trip)
fig

Figure(layout=FigureLayout(height='420px'))

In [128]:
# 8. To create a marker layer map between the four cities.
#  Combine the four city DataFrames into one DataFrame using the concat() function.
itinerary_df = pd.concat([vacation_start,vacation_stop1,vacation_stop2,vacation_stop3],ignore_index=True)
itinerary_df

,City_ID,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,268,Monrovia,LR,78.75,light rain,6.3005,-10.7969,Sunset Inn
1,324,Tabou,CI,78.35,light rain,4.4230,-7.3528,hotêl doufoulougou
2,420,Axim,GH,78.69,overcast clouds,4.8699,-2.2405,Axim Beach Resort And Spa
3,406,Porto Novo,BJ,82.22,overcast clouds,6.4965,2.6036,Tour Eiffel Hotel Benin


In [129]:
# 9 Using the template add city name, the country code, the weather description and maximum temperature for the city. 
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Description</dt><dd>{Current Description}</dd>
</dl>
"""

# 10a Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in itinerary_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = itinerary_df[["Lat", "Lng"]]

In [130]:
# 11a. Add a marker layer for each city to the map.
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))